In [1]:
from carla import Benchmark
import carla.evaluation.catalog as evaluation_catalog
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog
import shap 
from sklearn.preprocessing import OrdinalEncoder
from carla.data.causal_model import CausalModel
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from carla.data.catalog import CsvCatalog
import numpy as np 

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2


/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
import pandas as pd
# Load Data 
dataframe = pd.read_csv('./census.csv', index_col=['Unnamed: 0'])
continuous = dataframe.drop(columns=['target']).columns

dataset = CsvCatalog(file_path="census.csv",
                     continuous=continuous,
                     categorical=[],
                     immutables=[],
                     target='target')

In [3]:
#Model 

training_params = {"lr": 0.01, "epochs": 10, "batch_size": 16, "hidden_size": [18, 9, 2]}

ml_model = MLModelCatalog(
    dataset, model_type="ann", load_online=False, backend="pytorch"
)
ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)


balance on test set 0.24078624078624078, balance on test set 0.2408794988330672
Epoch 0/9
----------
train Loss: 0.5545 Acc: 0.7574

test Loss: 0.5530 Acc: 0.7591

Epoch 1/9
----------
train Loss: 0.5525 Acc: 0.7592

test Loss: 0.5522 Acc: 0.7591

Epoch 2/9
----------
train Loss: 0.5526 Acc: 0.7592

test Loss: 0.5525 Acc: 0.7591

Epoch 3/9
----------
train Loss: 0.5526 Acc: 0.7592

test Loss: 0.5537 Acc: 0.7591

Epoch 4/9
----------
train Loss: 0.5525 Acc: 0.7592

test Loss: 0.5531 Acc: 0.7591

Epoch 5/9
----------
train Loss: 0.5527 Acc: 0.7592

test Loss: 0.5531 Acc: 0.7591

Epoch 6/9
----------
train Loss: 0.5524 Acc: 0.7592

test Loss: 0.5525 Acc: 0.7591

Epoch 7/9
----------
train Loss: 0.5524 Acc: 0.7592

test Loss: 0.5525 Acc: 0.7591

Epoch 8/9
----------
train Loss: 0.5523 Acc: 0.7592

test Loss: 0.5561 Acc: 0.7591

Epoch 9/9
----------


In [ ]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:20]

display(test_factual)

,Unnamed: 0,Age,Workclass,Education-Num,Marital Status,...,Capital Gain,Capital Loss,Hours per week,Country,target
0,0,0.301370,0.875,0.800000,0.666667,...,0.02174,0.0,0.397959,0.951220,False
1,1,0.452055,0.750,0.800000,0.333333,...,0.00000,0.0,0.122449,0.951220,False
2,2,0.287671,0.500,0.533333,0.000000,...,0.00000,0.0,0.397959,0.951220,False
3,3,0.493151,0.500,0.400000,0.333333,...,0.00000,0.0,0.397959,0.951220,False
4,4,0.150685,0.500,0.800000,0.333333,...,0.00000,0.0,0.397959,0.121951,False
6,6,0.438356,0.500,0.266667,0.500000,...,0.00000,0.0,0.153061,0.560976,False
7,7,0.479452,0.750,0.533333,0.333333,...,0.00000,0.0,0.448980,0.951220,True
10,10,0.273973,0.500,0.600000,0.333333,...,0.00000,0.0,0.806122,0.951220,True
11,11,0.178082,0.875,0.800000,0.333333,...,0.00000,0.0,0.397959,0.463415,True
12,12,0.082192,0.500,0.800000,0.666667,...,0.00000,0.0,0.295918,0.951220,False


In [ ]:
hyperparams = {"loss_type": "BCE", "binary_cat_features": True}

recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)
df_cfs = recourse_method.get_counterfactuals(test_factual)

display(df_cfs)

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactua

,Age,Capital Gain,Capital Loss,Country,Education-Num,...,Occupation,Race,Relationship,Sex,Workclass
0,0.321253,0.041626,0.019884,0.971023,0.819885,...,0.091200,1.019861,0.180117,1.019876,0.855121
1,0.433441,0.019730,0.019524,0.969146,0.819586,...,0.268744,1.019357,-0.019814,1.019600,0.731458
2,0.327238,0.039574,0.039561,0.973703,0.572905,...,0.389423,0.960569,0.160431,1.039407,0.460457
3,0.444136,0.049175,0.049167,1.000205,0.449170,...,0.425017,0.549104,-0.049163,1.049120,0.450853
4,0.170553,0.019870,0.019867,0.141789,0.819868,...,0.694727,0.519853,1.019867,0.019796,0.480160
6,0.531572,0.109336,0.110005,0.518167,0.376740,...,0.656975,0.609418,0.102082,0.107207,0.390516
7,0.449858,0.029691,0.029686,0.980796,0.563021,...,0.286764,1.029648,-0.029684,1.029658,0.720326
10,0.293862,0.019742,0.019774,0.970975,0.619771,...,0.266085,0.519780,-0.019705,1.019730,0.480168
11,0.196203,0.019219,0.019444,0.481628,0.819097,...,0.724943,0.268447,-0.019749,1.018087,0.855502
12,0.141745,0.059902,0.059928,1.010545,0.859933,...,0.095980,1.059632,0.656273,0.059933,0.440109


In [ ]:
mapping_dict={
    "u1": "Sex",
    "u2": "Race",
    "u3": "Age",
    "u4": "Relationship",
    "u5": "Hours per week",
    "u6": "Capital Gain",
    "u7": "Education-Num",
    "u8": "Occupation",
    "u9": "target",}


In [ ]:
def get_pred_from_causal(scm,values,cf_label, mapping_dict):
    values['target']=cf_label
    def _get_noise_string(node):
        def _get_node_id(node):
            return node[1:]
        if not node[0] == "x":
            raise ValueError
        return "u" + _get_node_id(node)
    exogenous_variables = np.concatenate(
        [
            np.array(values[mapping_dict[node]]).reshape(-1, 1)
            for node in scm.get_topological_ordering("exogenous")
        ],
        axis=1,
    )
    exogenous_variables = pd.DataFrame(
        exogenous_variables, columns=scm.get_topological_ordering("exogenous")
    )

    endogenous_variables = exogenous_variables.copy()
    endogenous_variables = endogenous_variables.rename(
        columns=dict(
            zip(
                scm.get_topological_ordering("exogenous"),
                scm.get_topological_ordering("endogenous"),
            )
        )
    )
    # used later to make sure parents are populated when computing children
    endogenous_variables.loc[:] = np.nan
    for node in scm.get_topological_ordering("endogenous"):
        parents = scm.get_parents(node)
        if endogenous_variables.loc[:, list(parents)].isnull().values.any():
            raise ValueError(
                "parents in endogenous_variables should already be occupied"
            )
        endogenous_variables[node] = scm.structural_equations_np[node](
            exogenous_variables[_get_noise_string(node)],
            *[endogenous_variables[p] for p in parents],
        )
    print(endogenous_variables)
    labels=endogenous_variables['x9']
    return labels

In [ ]:
from carla.data import causal_model
from carla.evaluation import remove_nans
from carla.evaluation.api import Evaluation
class Sematic(Evaluation):
    """
    """

    def __init__(self, ml_model, causal_graph):
        self.ml_model= ml_model
        self.causal_graph=causal_graph
    def get_evaluation(self,factual: np.ndarray, counterfactual: np.ndarray, causal_model):
        # generate data 
        cf_label=self.ml_model.predict(np.array(counterfactual.values).reshape(1,-1))
        cf_label=np.argmax(cf_label)
        causal_label=get_pred_from_causal(self.causal_graph,counterfactual,cf_label, mapping_dict)
        print(cf_label)
        if cf_label ==causal_label:
            return 1 
        else: 
            return 0

class path_checker():
    pass

scm = CausalModel("adult")
print(scm.structural_equations_np)
metric = Sematic(ml_model,scm)
result=metric.get_evaluation(np.array([]),df_cfs.iloc[0],mapping_dict)
''' 
Analysis
Probitlity Distributions 
Histogram True Positive, False Positives 
'''
'''
Parts of counterfactuals actually change 
1hop and 2 hop relations 
'''
'''Histogram Features / true Positive / false Positive'''
# first initialize the benchmarking class by passing
# black-box-model, recourse method, and factuals into it
#benchmark = Benchmark(ml_model, recourse_method, factuals)

# now you can decide if you want to run all measurements
# or just specific ones.
#evaluation_measures = [
#    evaluation_catalog.YNN(benchmark.mlmodel, {"y": 5, "cf_label": 1}),
#    evaluation_catalog.Distance(benchmark.mlmodel),
#    evaluation_catalog.SuccessRate(),
#    evaluation_catalog.Redundancy(benchmark.mlmodel, {"cf_label": 1}),
#    evaluation_catalog.ConstraintViolation(benchmark.mlmodel),
#    evaluation_catalog.AvgTime({"time": benchmark.timer}),
#]

# now run all implemented measurements and create a
# DataFrame which consists of all results
#results = benchmark.run_benchmark(evaluation_measures)


Adult Model Initiate
Structural Equation Finished
Noise Distribution Finished
Finished Loading
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
{'x1': <function adult_model.<locals>.<lambda> at 0x7fe0afe21710>, 'x2': <function adult_model.<locals>.<lambda> at 0x7fe0afe219e0>, 'x3': <function adult_model.<locals>.<lambda> at 0x7fe0afe21a70>, 'x4': <function adult_model.<lo

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().